In [40]:
# default_exp data_block
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
from nbdev.export2html import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# Data Block

> Primary handlers for interfacing the openai gym envs

In [102]:
# export
from fastai.basic_data import *
from fastai.basic_train import *
from fastai.torch_core import *
from fastai.callbacks import *
from fastrl.wrappers import *
from fastrl.basic_agents import *
from fastrl.metrics import *
from dataclasses import asdict
from functools import partial
from fastprogress.fastprogress import IN_NOTEBOOK
from fastcore.utils import *
from fastcore.foundation import *
import torch.multiprocessing as mp
from functools import wraps
from queue import Empty
import textwrap
import logging
import gym

logging.basicConfig(format='[%(asctime)s] p%(process)s line:%(lineno)d %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S')
_logger=logging.getLogger(__name__)

In [43]:
# hide
_logger.setLevel('INFO')
import sys

## Dataset

`Dataset` instances are going to be a little different from the typically classification dataset that you might use in pytorch. Commonly, datasets have:
- A known size to iter through
- Maintain their state during the training sequence
- Randomly sample their dataset
- Have a common `x`/`y` or `input`/`target` data format

For our `ExperienceSourceDataset`, most of this is going to be different. 
- We can have multiple sources (envs)

You could think of a traditional dataset approach as being a mix of a `ExperienceSourceDataset` and a form of `ExperienceReplay`.

In [44]:
# export
def fix_s(x):
    "Flatten `x` to `(1,-1)` where `1` is the batch dim (B) unless **seems** to be an image e.g. has 3 dims (W,H,C), then it will attempt (B,W,H,C)."
    return (x if x.shape[0]==1 else 
            x.reshape(1,-1) if len(x.shape)==2 else
            np.expand_dims(x,0))

@dataclass
class Experience(object):
    s:np.array
    sp:np.array
    a:np.array
    r:np.array
    d:np.array
    agent_s:np.array
        
    # TODO possibly have x,y for more generic experience integration with datasets.
        
#     def __post_init__(self):
#         for k,v in asdict(self).items():
#             setattr(self,k,fix_s(v) if k in ['s','sp'] else np.array(v,dtype=float).reshape(1,-1))

In [45]:
exp=Experience(s=np.random.randint(1,3,(5,5)),
               sp=np.random.randint(1,3,(5,5)),
               a=np.random.randint(1,3,(1,2)),
               r=np.random.randint(1,3,(1,20)),
               d=np.random.randint(0,1,(5,5)),
               agent_s=np.random.randint(0,6,(1,5)))
asdict(exp)

{'s': array([[1, 2, 2, 2, 1],
        [2, 2, 1, 1, 1],
        [2, 1, 2, 2, 1],
        [2, 2, 2, 1, 2],
        [1, 2, 2, 1, 1]]),
 'sp': array([[2, 2, 1, 2, 1],
        [2, 1, 2, 2, 2],
        [1, 2, 2, 1, 1],
        [2, 2, 1, 1, 1],
        [2, 2, 2, 2, 1]]),
 'a': array([[2, 1]]),
 'r': array([[1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2]]),
 'd': array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]),
 'agent_s': array([[3, 2, 1, 3, 3]])}

## Datasets

We have a couple constraints on what the `ExperienceSourceDataset` is going to look like. For `__getitem__` it should return a tuple of 2 elements:

- `xb` which are the elements that are going to be directly fed into the model for getting actions.
- `yb` which are all of the elements provided by the environment that we can use for training.

Some semantics:
- `iter` means what is returned from `next(dataset)` for farthur what is returned from `[o for o in dataset]`.
- `step` means an individual step in the env. You can have multiple steps per `iter`.
- `batch_size` is the same thing as `step`.

In [99]:
# export
class ExperienceSourceCallback(LearnerCallback):
    def on_train_begin(self,*args,**kwargs):
        self.learn.data.train_dl.dataset.learn=self.learn
        if not self.learn.data.empty_val:
            self.learn.data.valid_dl.dataset.learn=self.learn

class ExperienceSourceDataset(Dataset):
    "Similar to fastai's `LabelList`, iters in-order samples from `1->len(envs)` `envs`."
    def __init__(self,env:str,n_envs=1,skip_n_steps=1,max_steps=None,pixels=False):
        def make_env():
            env=gym.make("CartPole-v1")
            if pixels:env.reset()
            return env
            
        self.envs=[make_env() for _ in range(n_envs)]
        if pixels:self.envs=[PixelObservationWrapper(e) for e in self.envs]
        self.learn=None
        self.callback_fns=[ExperienceSourceCallback]
        # Behavior modification fields / param fields.
        self.skip_n_steps=skip_n_steps
        self.max_steps=max_steps
        # Env tracking fields
        self.per_env_d=np.zeros((len(self.envs),))+1
        self.per_env_s=np.zeros((len(self.envs),*self.envs[0].observation_space.sample().shape))
        self.per_env_steps=np.zeros((len(self.envs)))
        self.per_env_rewards=np.zeros((len(self.envs)))
        # Metric fields
        self.total_rewards=[]
        self.total_steps=[]
        # Private funcational fields.
        self._step=0
        self._warned=False
        self._end_interation=False
        
    def __len__(self): return ifnone(self.max_steps,self.envs[0].spec.max_episode_steps)
    @log_args
    def is_last_step(self,d,steps,idx,max_steps)->bool:return bool(d) or steps>=max_steps-1
    @log_args
    def at_start(self,idx,steps):return idx==0 and steps==0
    @log_args
    def skip_loop(self,steps,skip_steps,d):return steps%skip_steps!=0 and not d
    @log_args
    def cycle_env(self,step,n_envs):self._step=0 if step==n_envs else step+1
    def stop_loop(self):
        self._end_interation=False
        raise StopIteration()
    @log_args
    def reset_envs(self,idx):
        for i,e in enumerate(self.envs):self.per_env_s[i]=e.reset() # TODO: There is the possiblity this will equal None (and crash)?
        self.per_env_steps=np.zeros((len(self.envs)),dtype=int)
        self.per_env_rewards=np.zeros((len(self.envs)))
    @log_args
    def pick_action(self,idx):
        if self.learn is None:
            if not self._warned:_logger.warning('`self.learn` is None. will use random actions instead.')
            self._warned=True
            return self.envs[0].action_space.sample(),np.zeros((1,1))
        return self.learn.predict(self.per_env_s[idx])
    @log_args
    def pop_total_rewards(self,total_rewards:List):
        if total_rewards:self.total_rewards,self.total_steps=[],[]
        return total_rewards
    @log_args
    def pop_reward_steps(self,total_rewards,total_steps):
        res=list(zip(total_rewards,total_steps))
        if res:self.total_r,self.total_iterations=[],[]
        return res
    
    @log_args
    def __getitem__(self,_):
        idx=self.inc  # This is the current env that we are running on
        if self._end_interation:self.stop_loop()
        if self.at_start(idx,self.per_env_steps[idx]):self.reset_envs(idx)
        
        while True:
            a,agent_s=self.get_action(idx)
            sp,r,d,_=self.envs[idx].step(a)
            if self.skip_loop(self.per_env_steps[idx],self.skip_n_steps,d):continue
            
            exp=Experience(self.s[idx],sp,a,r,d,agent_s=ifnone(agent_s,[]))
            
            self.per_env_rewards[idx]+=r
            self.per_env_s[idx]=sp
            self.per_env_steps[idx]+=1
            self.per_env_d[idx]=d
            
            if self.is_last_step(self.per_env_d[idx],self.per_env_steps[idx],idx,len(self)):
                self.total_rewards.append(self.per_env_rewards[idx])
                self.total_steps.append(self.per_env_steps[idx])
                self.per_env_steps[idx]=0
                
                self.cycle_env()
                self._end_interation=True
            return exp.s,asdict(exp)
    
#     @log_args
#     def __getitem__(self,_):
#         idx=self.inc  # This is the current env that we are running on
#         if self._end_interation:self.stop_loop()
#         if self.at_start(idx,self.per_env_steps[idx]):self.reset_envs(idx)
        
#         exps:List[Experience]=[]
#         while True:
#             a,agent_s=self.get_action(idx)
#             sp,r,self.d[idx],_=self.envs[idx].step(a)
#             if self.iterations[idx]%self.steps_delta==0 or self.d[idx]:
#                 exps.append(Experience(self.s[idx],sp,a,r,self.d[idx],agent_s=ifnone(agent_s,[])))
            
#             self.current_r[idx]+=r
#             self.s[idx]=sp
#             self.iterations[idx]+=1
#             if self.is_last_step(self.per_env_d[idx],self.per_env_steps[idx],idx,len(self)):
#                 self.total_r.append(self.current_r[idx])
#                 self.total_iterations.append(self.iterations[idx])
#                 self.iterations[idx]=0
                
#                 self.cycle_env()
#                 self._end_interation=True
#                 break
#             if len(exps)%self.steps==0:break
#         return [e.s for e in exps],[asdict(e) for e in exps]

add_docs(ExperienceSourceDataset,
         __init__='Provides an easy interface to iterate through an env or list of envs.\n Some importants notes:\n'
                  'an iteration through a loop. This is the maximum steps, and may be less due to the environment ending early.\n'
                  '- `skip_n_steps` are the number of steps to skip in the returned elements. This can be seen as frame skipping.',
        pick_action='Returns the action and learner\'s `self.learn.model`\'s state as determined by the learner for a state `self.s`'  
                    'belonging to env `idx`. If the `self.learn` is None, a random action using the `action_space` from `self.envs[0]` is used.\n\n'
                    'For the sake of clarity, the return type is Tuple[Tensor,Tensor] which can be understood as [a,agent_s] or [action, agent state].\n\n'
                    'While the returned action like `1`,`2` if discrete and `[0.2,0.5,0.2,0.1]` is a continuous action output, the agent state\n'
                    'is simply the raw values are were used to get the action. This information can be useful where for example we have an agent playing\n'
                    'the cartpole game. The action can either be `0` or `1`. This is considered the "action".\n'
                    'The agent state is the result of `self.learn.model(self.s)`. This is the raw `nn.Module` output and in the cartpole env case, is likely a\n'
                    'tensor of `Tensor([0.35,0.75])`, which for a discrete agent would an action of `1`. (we do an argmax, thus the action is which ever \n'
                    'neuron has the highest expected reward. This can be used to determine how confident the agent was when taking an action.',
        is_last_step='An env has reached it\'s last step when it is either `d` is true or `steps` is more than or equal to `self.max_steps-1`. '
                     'The reason this method has so many parameters is due to the use of `log_args` decorator. Having these all these parameters passed on makes '
                     'debugging much easier.',
        cycle_env='Increments the private `self._step` field to cyucle through the envs. Requires passing in parameters for easy debugging using `log_args`.',
        stop_loop='Raises the `StopIteration` exception and resets the `self._end_interation=False`. Used when the current env is done.',
        reset_envs='Performs a very interesting function for reseting all the envs. One might wonder "why not reset the envs individually?". The reason\n '
                   'is that performing a reset within a single process for most `gym` envs actually resets the overall renderer. This means that if you reset\n '
                   'one env, it affects all the others that haven\'t finished yet. In order to avoid this issue,\n '
                   'we only reset when all the envs are ready to be reset. The `idx` is here only for debugging.',
        at_start='Similar to `is_last_step`. We determine that the dataset as looped through all the envs and needs to reset them.\n '
                 'We have all the important params passed in for debugging via `log_args`.',
        pop_total_rewards='Returns and clears the `total_rewards` fields. Each element of total reward is a single episode or full iteration through an env.',
        pop_reward_steps='Returns and clears the `total_rewards` and `total_steps` fields. Each element in each represents data over a single episode.',
        skip_loop='If the current step should be skipped per `self.skip-step` and the current step is not done then we have the loop do a pass over.')

In [47]:
import sys
# make_env = lambda: gym.make("Acrobot-v1")
ds=ExperienceSourceDataset("CartPole-v1",n_envs=15,max_episode_step=50,steps=5)
dl=DataLoader(ds,batch_size=1,num_workers=0)
dones=[]
idx=0
for xb,yb in dl:
    dones.extend([o['d'] for o in yb])
    assert torch.cat(xb).shape[0]<=5 # Assert equal to the step size
    sys.stdout.flush()
    idx+=1
assert idx>2 # We hope that the env can at least loop twice

[08-02 20:13:24] p527 line:39 WARNING - `self.learn` is None. will use random actions instead.


In [50]:
import sys
# make_env = lambda: gym.make("Acrobot-v1")
ds=ExperienceSourceDataset("CartPole-v1",n_envs=3,max_episode_step=50,steps=5)
dl=DataLoader(ds,batch_size=1,num_workers=0)
dones=[]
for xb,yb in dl:
    dones.extend([o['d'] for o in yb])
    _logger.critical('xb: %s, yb: %s',torch.cat(xb).shape,yb[0].keys())
    assert torch.cat(xb).shape[0]<=1*3
    sys.stdout.flush()
assert(any([sum(i)>0 for i in dones]))
    
# make_env = lambda: gym.make("Acrobot-v1")
make_env = lambda: gym.make("CartPole-v1")
envs=[make_env() for _ in range(5)]
ds=ExperienceSourceDataset("CartPole-v1",n_envs=5,max_episode_step=50,steps=5)
dl=DataLoader(ds,batch_size=1,num_workers=0)
for xb,yb in dl:
    _logger.critical('xb: %s, yb: %s',torch.cat(xb).shape,yb[0].keys())
    assert torch.cat(xb).shape[0]<=5*5
    sys.stdout.flush()
    
lsr=ds.pop_total_r()
assert len(lsr)>1
assert np.average(lsr)>15

[08-02 20:28:53] p527 line:39 WARNING - `self.learn` is None. will use random actions instead.
[08-02 20:28:53] p527 line:8 CRITICAL - xb: torch.Size([5, 4]), yb: dict_keys(['s', 'sp', 'a', 'r', 'd', 'agent_s'])


AssertionError: 

In [ ]:
ds=ExperienceSourceDataset('CartPole-v1',n_envs=3,max_episode_step=50,steps=5,pixels=True)
dl=DataLoader(ds,batch_size=3,num_workers=0)
for xb,yb in dl:
    _logger.critical('xb: %s, yb: %s',torch.cat(xb).shape,yb[0].keys())
    assert torch.cat(xb).shape[0]<=5*3
    sys.stdout.flush()
ls_r_iter=ds.pop_r_interations()
assert len(ls_r_iter)>2
assert np.average([o[0] for o in ls_r_iter])>10
test_eq(int(sum([o[0] for o in ls_r_iter])),int(sum([o[1] for o in ls_r_iter])))

In [ ]:
# export
class FirstLastExperienceSourceDataset(ExperienceSourceDataset):
    "Similar to `ExperienceSourceDataset` but only keeps the first and last parts of a step. Can be seen as frame skipping."
    def __init__(self,*args,discount=0.99,**kwargs):
        super(FirstLastExperienceSourceDataset,self).__init__(*args,**kwargs)
        self.discount=discount
    
    @log_args
    def __getitem__(self,idx):
        s,exps=super(FirstLastExperienceSourceDataset,self).__getitem__(idx)
        exp=exps[-1]
        exp['s']=exps[0]['s']
        
        total_reward=0.0
        for e in reversed(exps):
            total_reward*=self.discount
            total_reward+=e['r']
        exp['r']=total_reward
        
        return [exps[0]['s']],[exp]

In [ ]:
ds=FirstLastExperienceSourceDataset("CartPole-v1",n_envs=3,max_episode_step=50,steps=5)
dl=DataLoader(ds,batch_size=3,num_workers=0)
index=0
for xb,yb in dl:
    _logger.critical('xb: %s, yb: %s',torch.cat(xb).shape,yb[0].keys())
    assert torch.cat(xb).shape[0]<=3
    sys.stdout.flush()
    index+=1

In [ ]:
# export
class AsyncExperienceSourceCallback(LearnerCallback):
    _order = -11
    
    def on_epoch_begin(self,**kwargs):
        ds=(self.learn.data.train_ds if self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        if not self.learn.data.empty_val:ds.pause_event.clear()

        if len(ds.data_proc_list)==0:
            if not hasattr(self.learn,'fitter'):
                _logger.warning('Using the default fitter function which will likely not work. Make sure your `AgentLearner` has a `fitter` attribute to actually run/train.')
            
            for proc_idx in range(ds.n_processes):
                _logger.info('Starting Process')
                data_proc=self.load_process()
                data_proc.start()
                ds.data_proc_list.append(data_proc)
                
    def load_process(self):raise NotImplementedError()
    def empty_queues(self):raise NotImplementedError()

    def on_batch_end(self,**kwargs):
        # If not training, pause train ds, otherwise pause valid ds
        ds=(self.learn.data.train_ds if not self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        if not self.learn.data.empty_val:ds.pause_event.set()
    
    def on_train_end(self,**kwargs):
        for ds in [self.learn.data.train_ds,None if self.learn.data.empty_val else self.learn.data.valid_ds]:
            if ds is None: continue
            ds.cancel_event.set()
            for _ in range(5):
                self.empty_queues()
            for proc in ds.data_proc_list: proc.join(timeout=0)
                
class AsyncGradExperienceSourceCallback(AsyncExperienceSourceCallback):
    def load_process(self):
        ds=(self.learn.data.train_ds if self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        return mp.Process(target=getattr(self.learn,'fitter',grad_fitter), 
                          args=(self.learn.model,self.learn.agent,ds.ds_cls),
                          kwargs={'grad_queue':ds.grad_queue,'loss_queue':ds.loss_queue,'pause_event':ds.pause_event,'cancel_event':ds.cancel_event,
                                  'metric_queue':ds.metric_queue})
    def empty_queues(self):
        ds=(self.learn.data.train_ds if self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        while not ds.grad_queue.empty(): ds.grad_queue.get()
        while not ds.loss_queue.empty(): ds.loss_queue.get()

class AsyncDataExperienceSourceCallback(AsyncExperienceSourceCallback):
    def load_process(self):
        ds=(self.learn.data.train_ds if self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        return mp.Process(target=getattr(self.learn,'fitter',data_fitter), 
                          args=(self.learn.model,self.learn.agent,ds.ds_cls),
                          kwargs={'data_queue':ds.data_queue,'pause_event':ds.pause_event,'cancel_event':ds.cancel_event,
                                  'metric_queue':ds.metric_queue})
    def empty_queues(self):
        ds=(self.learn.data.train_ds if self.learn.model.training or self.learn.data.empty_val else
            self.learn.data.valid_ds)
        while not ds.data_queue.empty():ds.data_queue.get()

In [ ]:
# export
def safe_fit(f):
    @wraps(f)
    def wrap(*args,cancel_event,**kwargs):
        try:
            return f(*args,cancel_event=cancel_event,**kwargs)
        finally:
            cancel_event.set()
            for k,v in kwargs.items():
                if k.__contains__('queue') and v is not None:v.put(None)
            return None
    return wrap

@safe_fit
def grad_fitter(model:nn.Module,agent:BaseAgent,ds:ExperienceSourceDataset,grad_queue:mp.JoinableQueue,
                loss_queue:mp.JoinableQueue,pause_event:mp.Event,cancel_event:mp.Event,metric_queue:mp.JoinableQueue=None):
    "Updates a `train_queue` with `model.parameters()` and `loss_queue` with the loss. Note that this is only an example grad_fitter."
    while not cancel_event.is_set(): # We are expecting the  grad_fitter to loop unless cancel_event is set
        cancel_event.wait(0.1)
        grad_queue.put(None)         # Adding `None` to `train_queue` will trigger an eventual ending of training
        loss_queue.put(None)
        if pause_event.is_set():     # There needs to be the ability for the grad_fitter to pause e.g. if waiting for validation to end.
            cancel_event.wait(0.1)   # Using cancel_event to wait allows the main process to end this Process.
        break

@safe_fit
def data_fitter(model:nn.Module,agent:BaseAgent,ds:ExperienceSourceDataset,data_queue:mp.JoinableQueue,
                pause_event:mp.Event,cancel_event:mp.Event,metric_queue:mp.JoinableQueue=None):
    _logger.warning('Using the `test_fitter` function. Make sure your `AgentLearner` has a `data_fitter` to actually run/train.')
    while not cancel_event.is_set(): # We are expecting the  grad_fitter to loop unless cancel_event is set
        cancel_event.wait(0.1)
        data_queue.put(None)         # Adding `None` to `train_queue` will trigger an eventual ending of training
        if pause_event.is_set():     # There needs to be the ability for the grad_fitter to pause e.g. if waiting for validation to end.
            cancel_event.wait(0.1)   # Using cancel_event to wait allows the main process to end this Process.     
            
def _soft_queue_get(q:mp.Queue,e:mp.Event):
    entry=None
    while not e.is_set():
        try:
            entry=q.get_nowait()
            break
        except Empty:pass
    return entry
            
class AsyncGradExperienceSourceDataset(ExperienceSourceDataset):
    "Contains dataloaders of multiple sub-datasets and executes them using `n_processes`. `xb` is the gradients from the agents, `yb` is the loss."
    def __init__(self,env_name:str,n_envs=1,ds_cls=ExperienceSourceDataset,max_episode_step=None,n_processes=1,queue_sz=None,*args,**kwargs):
        self.n_processes=n_processes
        self.n_envs=n_envs
        self.env_name=env_name
        self.ds_cls=ds_cls
        self.pause_event=mp.Event()                               # If the event is set, then the Process will freeze.
        self.cancel_event=mp.Event()                              # If the event is set, then the Process will freeze.
        self.max_episode_step=max_episode_step
        self.queue_sz=ifnone(queue_sz,self.n_processes*n_envs)
        self.grad_queue=mp.JoinableQueue(maxsize=self.queue_sz)
        self.loss_queue=mp.JoinableQueue(maxsize=self.queue_sz)
        self.metric_queue:mp.JoinableQueue=None
        self.data_proc_list=[]
        self.callback_fns=[AsyncGradExperienceSourceCallback]
        self._env=gym.make(self.env_name)
    
    def __len__(self): return ifnone(self.max_episode_step,self._env.spec.max_episode_steps)*self.n_envs
    
        
    def __getitem__(self,idx):
        if len(self.data_proc_list)==0: raise StopIteration()
        train_entry=_soft_queue_get(self.grad_queue,self.cancel_event)

        if train_entry is None:
            raise StopIteration()
        
        train_loss_entry=_soft_queue_get(self.loss_queue,self.cancel_event)
        return train_entry,[train_loss_entry]     
    
class AsyncDataExperienceSourceDataset(ExperienceSourceDataset):
    "Contains dataloaders of multiple sub-datasets and executes them using `n_processes`. `xb` is the gradients from the agents, `yb` is the loss."
    def __init__(self,env_name:str,n_envs=1,ds_cls=ExperienceSourceDataset,max_episode_step=None,n_processes=1,queue_sz=None,**kwargs):
        self.n_processes=n_processes
        self.n_envs=n_envs
        self.ds_cls=ds_cls
        self.env_name=env_name
        self.pause_event=mp.Event()                               # If the event is set, then the Process will freeze.
        self.cancel_event=mp.Event()                              # If the event is set, then the Process will freeze.
        self.max_episode_step=max_episode_step
        self.queue_sz=ifnone(queue_sz,self.n_processes*n_envs)
        self.data_queue=mp.JoinableQueue(maxsize=self.n_processes)
        self.metric_queue:mp.JoinableQueue=None
        self.data_proc_list=[]
        self.callback_fns=[AsyncDataExperienceSourceCallback]
        self._env=gym.make(self.env_name)
        
    def __len__(self): return ifnone(self.max_episode_step,self._env.spec.max_episode_steps)*self.n_envs
        
    def __getitem__(self,idx):
        if len(self.data_proc_list)==0: raise StopIteration()
        train_entry=_soft_queue_get(self.data_queue,self.cancel_event)

        if train_entry is None:
            raise StopIteration()

        return [e['s'] for e in train_entry],train_entry

In [ ]:
add_docs(AsyncGradExperienceSourceDataset,
         __init__=textwrap.fill("""The `AsyncGradExperienceSourceDataset` class is instantiated via passing 
         the `env_name` that we want to train on, and a `partial` class of a `ExperienceSourceDataset` called `ds_cls`."""))

## Dataset Shower
We can define a wrapper around a dataset which will show up to `rows * cols` environments.

In [ ]:
# export
if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

In [ ]:
%matplotlib inline

In [ ]:
# export
class DatasetDisplayWrapper(object):
    def __init__(self,ds,rows=2,cols=2,max_w=800):
        "Wraps a ExperienceSourceDataset instance showing multiple envs in a `rows` by `cols` grid in a Jupyter notebook."
        # Ref: https://stackoverflow.com/questions/1443129/completely-wrap-an-object-in-python
        # We are basically Wrapping any instance of ExperienceSourceDataset (kind of cool right?)
        clss=(ExperienceSourceDataset,FirstLastExperienceSourceDataset,
              AsyncGradExperienceSourceDataset,AsyncDataExperienceSourceDataset)
        assert issubclass(ds.__class__,clss),'Currently this only works with the ExperienceSourceDataset and Async*ExperienceSourceDataset class only.'
        self.__class__ = type(ds.__class__.__name__,(self.__class__, ds.__class__),{})
        self.__dict__=ds.__dict__
        self.rows,self.cols,self.max_w=rows,cols,max_w
        self.current_display=None
        if not IN_NOTEBOOK: 
            _logger.warning('It seems you are not running in a notebook. Nothing is going to be displayed.')
            return
        
        if self.envs[0].render('rgb_array') is None: self.envs[0].reset()
        rdr=self.envs[0].render('rgb_array')
        if rdr.shape[1]*self.cols>max_w:
            _logger.warning('Max Width is %s but %s*%s is greater than. Decreasing the number of cols to %s, rows increase by %s',
                            max_w,rdr.shape[1],self.cols,max_w%rdr.shape[1],max_w%rdr.shape[1])
            self.cols=max_w%rdr.shape[1]
            self.rows+=max_w%rdr.shape[1]
        self.max_displays=self.cols*self.rows
        self.current_display=np.zeros(shape=(self.rows*rdr.shape[0],self.cols*rdr.shape[1],rdr.shape[2])).astype('uint8')
        _logger.info('%s, %s, %s, %s, %s',0,0//self.cols,0%self.cols,rdr.shape,self.current_display.shape)

    def __getitem__(self,idx):
        o=super(DatasetDisplayWrapper,self).__getitem__(idx)
        idx=idx%self.max_displays
        if self.current_display is not None and idx<self.rows*self.cols:
            display.clear_output(wait=True)
            im=self.envs[idx].render(mode='rgb_array')
            self.current_display[(idx//self.cols)*im.shape[0]:(idx//self.cols)*im.shape[0]+im.shape[0],
                                 (idx%self.cols)*im.shape[1]:(idx%self.cols)*im.shape[1]+im.shape[1],:]=im
            new_im=PIL.Image.fromarray(self.current_display)
            display.display(new_im)
        else:
            display.display(PIL.Image.fromarray(self.current_display))
        return o

## DataBunch

In [ ]:
# export
class ExperienceSourceDataBunch(DataBunch):
    @classmethod
    def from_env(cls,env:str,n_envs=1,firstlast=False,display=False,max_steps=None,skip_step=1,path:PathOrStr='.',add_valid=True,
                 cols=1,rows=1,max_w=800):
        def create_ds(make_empty=False):
            _ds_cls=FirstLastExperienceSourceDataset if firstlast else ExperienceSourceDataset
            make_env = lambda: gym.make(env)
            envs=[make_env() for _ in range(n_envs)]
            _ds=_ds_cls(envs,max_episode_step=0 if make_empty else max_steps,steps=skip_step)
            if display:_ds=DatasetDisplayWrapper(_ds,cols=cols,rows=rows,max_w=max_w)
            return _ds
            
        dss=(create_ds(),create_ds(not add_valid))
        return cls.create(*dss,bs=n_envs,num_workers=0)
    
    @classmethod
    def create(cls, train_ds:Dataset, valid_ds:Dataset, test_ds:Optional[Dataset]=None, path:PathOrStr='.', bs:int=64,
               val_bs:int=None, num_workers:int=defaults.cpus, dl_tfms:Optional[Collection[Callable]]=None,
               device:torch.device=None, collate_fn:Callable=data_collate, no_check:bool=False, **dl_kwargs)->'DataBunch':
        "Create a `DataBunch` from `train_ds`, `valid_ds` and maybe `test_ds` with a batch size of `bs`. Passes `**dl_kwargs` to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        dls = [DataLoader(d, b, shuffle=s, drop_last=s, num_workers=num_workers, **dl_kwargs) for d,b,s in
               zip(datasets, (bs,val_bs,val_bs,val_bs), (False,False,False,False)) if d is not None]
        return cls(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [ ]:
data=ExperienceSourceDataBunch.from_env('CartPole-v1',n_envs=5,display=False,firstlast=False,add_valid=False)
for xb,yb in data.train_dl:
    test_eq(len(xb),1)
    test_eq(tuple(xb[0].shape),(5,4))

In [ ]:
data=ExperienceSourceDataBunch.from_env('CartPole-v1',n_envs=5,display=False,firstlast=True,add_valid=False)
for xb,yb in data.train_dl:
    test_eq(len(xb),1)
    test_eq(tuple(xb[0].shape),(5,4))

In [ ]:
# export
class AsyncExperienceSourceDataBunch(ExperienceSourceDataBunch):
    @classmethod
    def from_env(cls,env:str,n_envs=1,data_exp=True,firstlast=False,display=False,max_steps=None,skip_step=1,path:PathOrStr='.',add_valid=True,
                 cols=1,rows=1,max_w=800,n_processes=1,queue_sz=None):
        def create_ds(make_empty=False):
            _sub_ds_cls=FirstLastExperienceSourceDataset if firstlast else ExperienceSourceDataset
            _sub_ds_cls=partial(_sub_ds_cls,env=env,n_envs=1,max_episode_step=0 if make_empty else max_steps,steps=skip_step)
            _ds_cls=AsyncDataExperienceSourceDataset if data_exp else AsyncGradExperienceSourceDataset
            _ds=_ds_cls(env,max_episode_step=0 if make_empty else max_steps,steps=skip_step,ds_cls=_sub_ds_cls,n_processes=n_processes,queue_sz=queue_sz)
            for k,v in {'env':env,'n_envs':n_envs,'steps':skip_step}.items():
                if not hasattr(_ds,k):setattr(_ds,k,v)
            if display:_ds=DatasetDisplayWrapper(_ds,cols=cols,rows=rows,max_w=max_w)
            return _ds
            
#         dss=(create_ds(),create_ds() if add_valid else None)
        return cls.create(create_ds(),create_ds(not add_valid),bs=n_envs,num_workers=0)

In [ ]:
# export
@safe_fit
def dqn_fitter(model:nn.Module,agent:BaseAgent,ds:ExperienceSourceDataset,data_queue:mp.JoinableQueue,
               pause_event:mp.Event,cancel_event:mp.Event,metric_queue:mp.JoinableQueue=None):
    dataset=ds()
    while not cancel_event.is_set(): 
        for xb,yb in dataset:
            data_queue.put(yb)
            if pause_event.is_set():cancel_event.wait(0.1) 
            if cancel_event.is_set():break
        if metric_queue is not None:metric_queue.put(TotalRewards(np.mean(dataset.pop_total_r())))
        if cancel_event.is_set():break
                
@safe_fit
def dqn_grad_fitter(model:nn.Module,agent:BaseAgent,ds:ExperienceSourceDataset,grad_queue:mp.JoinableQueue,loss_queue:mp.JoinableQueue,
                    pause_event:mp.Event,cancel_event:mp.Event,metric_queue:mp.JoinableQueue=None):
    dataset=ds()
    while not cancel_event.is_set(): 
        for xb,yb in dataset:
            sys.stdout.flush()
            grad_queue.put(xb)
            loss_queue.put(0.5)
            if pause_event.is_set():cancel_event.wait(0.1) 
            if cancel_event.is_set():break
        if metric_queue is not None:metric_queue.put(TotalRewards(np.mean(dataset.pop_total_r())))
        if cancel_event.is_set():break

@safe_fit
def buggy_dqn_fitter(model:nn.Module,agent:BaseAgent,ds:ExperienceSourceDataset,data_queue:mp.JoinableQueue,
                pause_event:mp.Event,cancel_event:mp.Event,metric_queue:mp.JoinableQueue=None):
    dataset=ds()
    while not cancel_event.is_set(): 
        for xb,yb in dataset:
            data_queue.put(yb)
            if pause_event.is_set():cancel_event.wait(0.1) 
            if cancel_event.is_set():break
            raise Exception('Crashing on purpose')
        if cancel_event.is_set():break

In [ ]:
from fastrl.basic_train import *
import pytest

If no fitter is added, then calling the fit function will likelly result in a `TypeError` where the `smooth_loss` is missing.

In [ ]:
class FakeRunCallback2(LearnerCallback):
    def on_backward_begin(self,*args,**kwargs): return {'skip_bwd':True,'skip_validate':True}
#     def on_batch_begin(self,last_target,**kwargs):
#         print([o['d'] for o in last_target])
#         sys.stdout.flush()

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',data_exp=True,display=False,max_steps=50,firstlast=False,add_valid=False,n_processes=1,n_envs=1)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback2])
setattr(learn,'fitter',dqn_fitter)
learn.fit(2,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',data_exp=True,display=False,max_steps=50,firstlast=True,add_valid=False,n_processes=1,n_envs=1)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback])
with pytest.raises(TypeError):
    learn.fit(1,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',data_exp=False,display=False,firstlast=True,add_valid=False,n_processes=2,n_envs=2)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback])
setattr(learn,'fitter',dqn_grad_fitter)
learn.fit(1,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',display=False,firstlast=True,add_valid=False,max_steps=50,n_processes=2,n_envs=2)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback])
setattr(learn,'fitter',dqn_fitter)
learn.fit(1,lr=0.01,wd=1)

In [ ]:
import pytest
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',display=False,firstlast=True,add_valid=False,max_steps=50,n_processes=2,n_envs=2)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback])
setattr(learn,'fitter',buggy_dqn_fitter)
with pytest.raises(TypeError):
    learn.fit(1,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',display=False,firstlast=True,add_valid=False,max_steps=50,n_processes=2,n_envs=2)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=DQNAgent(model=model)
learn=AgentLearner(data,model,agent=agent,callback_fns=[FakeRunCallback])
setattr(learn,'fitter',dqn_fitter)
learn.fit(1,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',n_envs=5,display=False,max_steps=50,firstlast=False,add_valid=False)
for xb,yb in data.train_dl:
    test_eq(len(xb),1)
    test_eq(tuple(xb[0].shape),(5,4))

In [103]:
# hide
from nbdev.export import *
notebook2script()
notebook2html(n_workers=0)

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 02_callbacks.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_metrics.ipynb.
Converted 05_data_block.ipynb.
Converted 06_basic_train.ipynb.
Converted 12_a3c.a3c_data.ipynb.
Converted index.ipynb.
Converted notes.ipynb.
converting: /opt/project/fastrl/nbs/12_a3c.a3c_data.ipynb
converting: /opt/project/fastrl/nbs/06_basic_train.ipynb
converting: /opt/project/fastrl/nbs/05_data_block.ipynb
